In [121]:
%cd /home/richard_fernandes/Downloads/sentinelsat-main

/home/richard_fernandes/Downloads/sentinelsat-main


1.  Parse csv file to extract ALL image names, even multiple entries
        parse data as csv
        extract the column as a list of lists
        flatten the lixt of lists into one big list
        remove null none 
2.  make pandas df from clean list with one column correspondng to input image name
3.  write function to search for the input image name in gee and return system:id , put in new column of pandas df
        start gee api
        in python for loop of df column of image names
            use gee command to filter the S2 collection to get image 
            real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index","20210719T190919_20210719T191700_T10TER")) \
                         .first() \
                         .get('PRODUCT_ID') \
                         .getInfo()
4.  iterate over list of real names
    

In [122]:
#import libraries
import ee
import pandas as pd
import os

# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [123]:
#Autheticate and initillize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWtgzh5lUd0y1q2J01oMGOIS2Mnn7_Elf9egqdey0yqD8LmYt8is_4PKtGM



Successfully saved authorization token.


In [125]:
def directory1(directory):
    """Gets the names of a directory"""
    entries = os.listdir(directory)
    return entries
    # print(entries)  

In [170]:
def list_splitter(data):
    """cleans/splits df of list of images"""
    
    # Input data
    df = pd.read_csv(data)
    # print(df)
    
    # Change type to string, remove [], send to list
    df['Images'] = df['Images'].astype(str)
    # print(df['Images'])
    df['Images'] = df['Images'].map(lambda x: x.lstrip('[').rstrip(']'))
    list_of_img =  df["Images"].tolist() 
    # print(list_img)
    
    # Clean list without nan
    cleanedList = [x for x in list_of_img if str(x) != 'nan']
    
    # Print list
    # print("list1")
    # print(cleanedList)
    
    # print("#---------------------------")
    # print(len(cleanedList))
    
    # Arrays
    cleanedList2  = []
    list_to_add = []
    
    # Find list images containing more than 1 img
    for counter0 in range(0,len(cleanedList)):
        count_comma = cleanedList[counter0].count(',')
        # print(count_comma)
       
        if count_comma>0:
            list_multiple  = cleanedList[counter0]
            # print(list_multiple)
            cleanedList2.append(list_multiple)
            
    # Find list images containing 1 img  
    cleanedList3 = [item for item in cleanedList if item not in cleanedList2]
    
    # Print
    # print("clean list2")
    # print(cleanedList2)
    # print("clean list3")
    # print(cleanedList3)

    # Split the data into separate imgs 
    for counter1 in range(0,len(cleanedList2)):
        list_item =cleanedList2[counter1] 
        # print(list_item)
        mylist = list_item.split(',')
        for counter2 in range(0,len(mylist)):
            withoutspace  = mylist[counter2].replace(" ","")
            
            list_to_add.append(withoutspace)
    
    
    # print("list to add")
    # print(list_to_add)
    
    
    # add two lists      
    list_fin = list_to_add+cleanedList3
    
    # create dataframe
    newdf= pd.DataFrame(list_fin,columns=["Images__clean"])
    newdf = newdf.drop_duplicates()

    # newdf.to_csv("/home/richard_fernandes/Downloads/data1.csv")
    
    print("______________________________________________")
    return newdf

    
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/ABBY_kmlwithImages.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/DSNYCLEARIMAGES2.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/UNDECLEARIMAGES.csv"))

______________________________________________
                            Images__clean
0  20190606T165901_20190606T170333_T15TYM
1  20190606T165901_20190606T170333_T16TCS
2  20200806T164901_20200806T165818_T15TYM
3  20200806T164901_20200806T165818_T16TCS


In [103]:
def real_name(short_name):
    """ function to help find real names of files"""
    real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index",short_name)) \
                    .first() \
                    .get('PRODUCT_ID') \
                    .getInfo()

    return real_name

In [40]:
# print(real_name("20210719T190919_20210719T191700_T10TER"))

S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339


In [83]:
def search_string(string1):
    """ searchs esa datahub for image and downsloads it """
    
    # Connect to the API
    api = SentinelAPI('****', '******', 'https://apihub.copernicus.eu/apihub')

    # Split string 
    string = string1 
    split_s=string.split("_")
    
    # Built a file string
    sat_name = "S2A"+"_"
    rest_name = split_s[1]+"_"+split_s[2]+"_"+split_s[3]+"_"+split_s[4]+"_"+split_s[5]+"_"+split_s[6]
    safe_name = '.SAFE'
    # # wc = '_*_'

    #try and except to find errors
    try:
        print("#--------Try---------------------------------------")
        sat_name = 'S2A_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)

        query1 = api.query(filename = filename1)
        # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)
        # print(result)
        
        #check if something is downloaded
        if str(result) =="ResultTuple(downloaded={}, retrieval_triggered={}, failed={})":
            raise ValueError('Nothing was dowloaded')

    except:
        print("#--------Except---------------------------------------")
        sat_name ='S2B_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)
        
        query1 = api.query(filename = filename1)
        # # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)

    # print(result)
# 
# print(search_string("S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339"))

#--------Try---------------------------------------
#--------Except---------------------------------------


ResultTuple(downloaded={'1d58d0fd-108c-424c-a818-43d12545e305': {'id': '1d58d0fd-108c-424c-a818-43d12545e305', 'title': 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339', 'size': 741432504, 'md5': '7c8c1c3c7358bd01a8bd71627d69e9f3', 'date': datetime.datetime(2021, 7, 19, 19, 9, 19, 24000), 'footprint': 'POLYGON((-122.0253 46.04752322186573,-122.03458 46.02596045913228,-122.09686 45.88072405579696,-122.15697 45.73506781249301,-122.216995 45.589502397244495,-122.27751 45.44413955351746,-122.337585 45.2987002868348,-122.39809 45.15339736104352,-122.4362 45.061817282498176,-123.00026 45.06525999447365,-123.00026 46.05357436995249,-122.0253 46.04752322186573))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('1d58d0fd-108c-424c-a818-43d12545e305')/$value", 'Online': True, 'Creation Date': datetime.datetime(2021, 7, 20, 1, 18, 17, 594000), 'Ingestion Date': datetime.datetime(2021, 7, 20, 1, 18, 0, 554000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odat

In [ ]:
# List of csvs in directory
list_directory = directory1("/home/richard_fernandes/Downloads/S2_clearskyImage")
# print(list_directory)

# Looping through csvs
for counter0 in range(0,len(list_directory)): 
# for counter0 in range(0,2):  

    # Clean lists of imgs in csv
    newdf = list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/"+list_directory[counter0])
    # print(newdf)
    
    # Search every Image
    for counter in range(0,len(newdf.index)): 
        if ~newdf.empty:
            
            # Run through process
            # print("p")
            
            # Takes img name
            name_stage0 = newdf['Images__clean'][counter]
            # print(name_stage0)
            
            # Finds real name of img
            name_stage1 = real_name(name_stage0)
            
            #seaach and download img
            search_string(name_stage1)
            # # print(name_stage1)

['UNDECLEARIMAGES.csv', 'NOGPCLEARIMAGES.csv', 'DSNY_CLEARIMAGE.csv', 'MCDICLEARIMAGES.csv', 'SERCCLEARIMAGES.csv', 'DSNYCLEARIMAGES2.csv', 'ABBY_kmlwithImages.csv', 'SJERCLEARIMAGES.csv', 'STEI_CLEARImages.csv', 'LENOCLEARIMAGES.csv', 'HOPB_CLEARIMAGE.csv', 'JORNCLEARIMAGES.csv', 'WREFCLEARIMAGES.csv']
______________________________________________
#--------Try---------------------------------------


ResultTuple(downloaded={'77628b0f-ad5b-4d48-96f3-99c7c8621f3e': {'id': '77628b0f-ad5b-4d48-96f3-99c7c8621f3e', 'title': 'S2A_MSIL2A_20190606T165901_N0212_R069_T15TYM_20190606T212006', 'size': 989640508, 'md5': '00949050c7517f9a1fb5e550aeb80103', 'date': datetime.datetime(2019, 6, 6, 16, 59, 1, 24000), 'footprint': 'POLYGON((-88.93643 46.853205179309775,-88.97769 46.758338715445376,-89.04071 46.612956717610906,-89.10373 46.46761135641404,-89.166626 46.322343867367934,-89.22876 46.17708780681065,-89.290344 46.03187582600645,-89.34372 45.90542956297485,-90.42053 45.93643087306754,-90.373505 46.92357305830357,-88.93436 46.881459661637656,-88.93643 46.853205179309775))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('77628b0f-ad5b-4d48-96f3-99c7c8621f3e')/$value", 'Online': True, 'Creation Date': datetime.datetime(2019, 6, 7, 2, 51, 18, 98000), 'Ingestion Date': datetime.datetime(2019, 6, 7, 2, 50, 43, 579000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/P

LTA retrieval:   0%|                                                              | 0/1 [03:06<?, ?product/s]


#--------Except---------------------------------------
ResultTuple(downloaded={}, retrieval_triggered={}, failed={})
#--------Try---------------------------------------


LTA retrieval:   0%|                                                              | 0/1 [00:07<?, ?product/s]


...

LTA retrieval: 100%|████████████████████████████████████████████████████| 1/1 [34:14<00:00, 2054.76s/product]


MD5 checksumming:   0%|                                                          | 0.00/1.15G [00:00<?, ?B/s]
MD5 checksumming:   3%|█▋                                                | 39.6M/1.15G [00:00<00:02, 395MB/s]
MD5 checksumming:   8%|████                                              | 94.1M/1.15G [00:00<00:02, 484MB/s]
MD5 checksumming:  13%|██████▌                                            | 149M/1.15G [00:00<00:01, 515MB/s]
MD5 checksumming:  18%|█████████                                          | 204M/1.15G [00:00<00:01, 527MB/s]
MD5 checksumming:  22%|███████████▍                                       | 257M/1.15G [00:00<00:01, 511MB/s]
MD5 checksumming:  27%|█████████████▋                                     | 308M/1.15G [00:00<00:01, 495MB/s]
MD5 checksumming:  31%|████████████████                                   | 362M/1.15G [00:00<00:01, 510MB/s]
MD5 chec

ResultTuple(downloaded={'4333ad73-49e2-40cf-9e50-ca99bde3de00': {'id': '4333ad73-49e2-40cf-9e50-ca99bde3de00', 'title': 'S2B_MSIL2A_20210612T173909_N0300_R098_T14TLS_20210612T215521', 'size': 1150014482, 'md5': '1252a15e1e4a7645cb5f5efd42abb112', 'date': datetime.datetime(2021, 6, 12, 17, 39, 9, 24000), 'footprint': 'POLYGON((-101.627014 46.92356100851489,-100.1853 46.947573655919015,-100.16403 45.95962253249568,-101.57999 45.93641922924405,-101.627014 46.92356100851489))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('4333ad73-49e2-40cf-9e50-ca99bde3de00')/$value", 'Online': True, 'Creation Date': datetime.datetime(2021, 6, 13, 14, 12, 20, 104000), 'Ingestion Date': datetime.datetime(2021, 6, 13, 14, 11, 21, 125000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('4333ad73-49e2-40cf-9e50-ca99bde3de00')/Products('Quicklook')/$value", 'path': 'S2B_MSIL2A_20210612T173909_N0300_R098_T14TLS_20210612T215521.zip', 'downloaded_bytes': 0}}, retrieval_t

LTA retrieval: 100%|██████████████████████████████████████████████████| 1/1 [1:38:44<00:00, 5924.94s/product]


MD5 checksumming:   0%|                                                          | 0.00/1.15G [00:00<?, ?B/s]
MD5 checksumming:   4%|██▏                                               | 51.5M/1.15G [00:00<00:02, 515MB/s]
MD5 checksumming:   9%|████▌                                              | 103M/1.15G [00:00<00:02, 517MB/s]
MD5 checksumming:  13%|██████▉                                            | 156M/1.15G [00:00<00:01, 520MB/s]
MD5 checksumming:  18%|█████████▏                                         | 208M/1.15G [00:00<00:01, 511MB/s]
MD5 checksumming:  23%|███████████▋                                       | 264M/1.15G [00:00<00:01, 529MB/s]
MD5 checksumming:  27%|█████████████▉                                     | 317M/1.15G [00:00<00:01, 518MB/s]
MD5 checksumming:  32%|████████████████▎                                  | 369M/1.15G [00:00<00:01, 518MB/s]
MD5 chec

ResultTuple(downloaded={'8dae7076-65da-4d22-96e7-23c6deb1a6d1': {'id': '8dae7076-65da-4d22-96e7-23c6deb1a6d1', 'title': 'S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310', 'size': 1154841775, 'md5': '53f61d51e28308cbc152b791778f968f', 'date': datetime.datetime(2020, 7, 13, 17, 9, 1, 24000), 'footprint': 'POLYGON((-96.66684 39.72681533153163,-95.387634 39.69403311805837,-95.43793 38.70656329494251,-96.69937 38.73821864423846,-96.66684 39.72681533153163))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8dae7076-65da-4d22-96e7-23c6deb1a6d1')/$value", 'Online': True, 'Creation Date': datetime.datetime(2020, 7, 14, 3, 31, 4, 180000), 'Ingestion Date': datetime.datetime(2020, 7, 14, 3, 30, 18, 691000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8dae7076-65da-4d22-96e7-23c6deb1a6d1')/Products('Quicklook')/$value", 'path': 'S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310.zip', 'downloaded_bytes': 0}}, retrieval_triggered={

LTA retrieval:   0%|                                                              | 0/1 [00:00<?, ?product/s]